 # 3- KEŞIFSEL VERI ANALIZI (EDA) 

![veritablosu](https://i.pinimg.com/originals/6a/37/fb/6a37fbcea9be5ecf69925610576fcc84.png)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
import missingno as msno
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 3A-TRANSACTION VE IDENTITY DATASETLERINI TANIYALIM

#### Train Transaction
 590540 rows ve 394 columns dan olusuyor

**TransactionID :** 590540 islem icin 2987000 3577539 arasinda id verilmistir

**isFraud:**  0 normal  işlemi 1 ise dolandırıcılık işlemini temsil eder.

**TransactionDT:** tam belirli olmayan bir tarih saatinden referansla işlemlerin saniye cinsinden timedelta larını içerir. Daha anlamlı hale getirmek için zaman dilimlerine ayırıldığından 190 gunlük işlemleri içerdiği anlaşılıyor.

**TransactionAMT:** USD bazında işlem tutarları değerini içerir. noktadan sonra 3 decimal olan değerlerin başka doviz kurlarında yapılan işlemlerin USD ye değiştirme yapıldığı anlaşılıyor.

**ProductCD:** her işlem için ürün kodu değerlerini içerir - katogorik değişkenler
                w 71%, c 14%,  (h,s,r) 15% 

**card1, card2, card3, card5 :** kartın bankası, ülke vb. gibi ödeme kartı bilgileri.

**card4**  işlemin yapıldığı kredi kartı şirketidir - katogorik değişkenler
            visa 65% MasterCard 32% (discover, american express) 3%
            
**card6** işlemin yapıldığı kart türüdür - katogorik değişkenler
            debit 74% credit 25% (charge card, debit or credit) 1%


**addr1** alici faturalandırma bölgesi adresi
**addr2** alici faturalandırma ülkesi adresi


**dist1:**  59% Nan değeri olan fatura adresi, posta adresi, posta kodu, IP adresi, telefon alanı vb. arasındaki mesafeleri ifade eder.

**dist2:** 94% Nan değeri var silinebilir

**R_emaildomain** alıcı kişinin mail adresi - bağzı işlemlerde alıcı gerek olmadığından 76% Nan değeri var.

P_ and (R__) emaildomain: purchaser and recipient email domain
C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
D1-D15: timedelta, such as days between previous transaction, etc.d8-d9 float digerleri int degerler

M1-M9: match, such as names on card and address, etc.
             T F or NaN
Column M4 appears to be different with values like M2 and M0

V1-V339: Vesta engineered rich features, including ranking, counting, and other entity relations.

https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt

In [ ]:
# CALISACAGIM DATASET
train_transaction=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
test_transaction=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [ ]:
# test_transaction setinde isFraud sutunu bulunmadigindan concat oncesi yeni ekliyoruz ve 2 degerini dolduruyoruz
test_transaction['isFraud']=2
# train_transaction ve test_transaction setlerini concat ediyoruz
transaction=pd.concat([train_transaction, test_transaction], ignore_index=True)

In [ ]:
transaction['D1'].head(40)

In [ ]:
# Column ve Degerlerine genel bakis
for col, values in transaction.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

In [ ]:
# NE KADAR ROW-COLINM OLDUGU

print(f'Train Transaction has {transaction.shape[0]} rows and {transaction.shape[1]} columns.')

In [ ]:
#COLUMN ISIMLER
transaction.columns

In [ ]:
# BENIM KISIM OLAN COLUMNLARI BASKA BIR VARIABLE ATIYORUZ ve CALISMAMIZDA COPYASINI KULLANIYORUZ
transaction_mycolumns=transaction.iloc[:,0:55]
my_transaction=transaction_mycolumns.copy()

In [ ]:
my_transaction.info()

In [ ]:
my_transaction.info(memory_usage='deep')

In [ ]:
my_transaction.describe()

In [ ]:
# kololasyonu anlam ifade eden kisimlari gozlemleyelim
a=my_transaction.corr().abs()<1
b=my_transaction.corr().abs()>0.5
my_transaction.corr().abs()[a&b]


In [ ]:
# NE KADAR COLUMNDA MISSING VALUE OLDUGU
print(f'There are {my_transaction.isnull().any().sum()} columns in my_train transaction dataset with missing values.')

In [ ]:
# Eksik verileri oransal bar ile gorsellestirme
msno.bar(train_Mytransaction);

In [ ]:
msno.matrix(my_transaction);

In [ ]:
# degiskenler arasindaki yokluk koralasyonu  -1 demek- birinde null varsa digerinde de null her durumda var
msno.heatmap(my_transaction);

# Mapping emails

Tüm e-posta alanlarını ilgili kuruluşlara göre gruplayacağım. 300'den az girdiye sahip tüm değerleri "Other" olarak ayarlayacağım.

Company yahoo / ymail / frontier / rocketmail -> Yahoo

hotmail / outlook / live / msn -> Microsoft

icloud / mac / me -> Appe

prodigy / att / sbcglobal-> AT&T

centurylink / embarqmail -> Centurylink

aim / aol -> AOL

twc / charter -> Spectrum

gmail.com / gmail -> Google

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    my_transaction[c + '_bin'] = my_transaction[c].map(emails)
    my_transaction[c + '_suffix'] = my_transaction[c].map(lambda x: str(x).split('.')[-1])
    my_transaction[c + '_suffix'] = my_transaction[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')


In [ ]:
for column_name in ['P_emaildomain', 'R_emaildomain']:
    my_transaction.drop(column_name, axis=1, inplace=True)

-Kategorik degiskenler icin deger atama
*mod islemine gore-en sik raslanan katogorik degiskene gore deger atama

In [ ]:
object_columns_name=my_transaction.select_dtypes(include='object').columns

In [ ]:
object_columns_name

In [ ]:
for kategorik_sutun in object_columns_name:
    modun_stringdegeri=my_transaction[kategorik_sutun].mode()[0]
    my_transaction[kategorik_sutun].fillna(modun_stringdegeri,inplace=True)

# Kategorik Değerleri Dönüştürme (Frekans)

Değişken Donusumleri -Variable Transformation

In [ ]:
# Kategorik degerlerin eksik degerlerini doldurma yontemlerinden Frekansi kullanacagiz
def frekans(data,columns,n_label="NONE"):
    
    for col in columns:
        #data[col].fillna(n_label,inplace=True)
        fq_encode = data[col].value_counts(dropna=False).to_dict()   
        data[col+"_Fr"] = data[col].map(fq_encode)
        data=data.drop(col,axis=1)
    return data

In [ ]:
my_transaction = frekans(my_transaction,object_columns_name)

In [ ]:
my_transaction.head()

In [ ]:
my_transaction.info()

[](http://)# Yeni Öznitelikler Oluşturma (Feature Extraction)

In [ ]:
# Preprocess date column
START_DATE = '2017-12-01'
my_transaction = my_transaction.rename(columns={'TransactionDT': 'TransactionDate'})
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
my_transaction['TransactionDate'] = my_transaction['TransactionDate'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))


print(my_transaction['TransactionDate'].head())
print(my_transaction['TransactionDate'].tail())

In [ ]:
my_transaction.columns

# Aykırı/Uç Verileri (Outliers) Tespit Etme

# A-Veri Temizleme (Data Cleaning / Cleasing)
## 3- Outlier Detection- Aykiri gozlem Analizi

In [ ]:
import seaborn as sns
my_transaction_TransactionAmt_Fraud=my_transaction[['isFraud','TransactionAmt']]
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(my_transaction_TransactionAmt_Fraud.corr(), ax=ax,linewidths=.5,annot=True)
plt.show()

In [ ]:
# boxplot
sns.boxplot(x=my_transaction['TransactionAmt']);

In [ ]:
# boxplot da esik deger atama
Q1=my_transaction['TransactionAmt'].quantile(0.25)
Q3=my_transaction['TransactionAmt'].quantile(0.75)
IQR=Q3-Q1
my_TransactionAmt_Alt_Sinir = Q1-2.5*IQR
my_TransactionAmt_Ust_Sinir = Q3 + 2.5*IQR

In [ ]:
Q1, Q3 , IQR,my_TransactionAmt_Alt_Sinir,my_TransactionAmt_Ust_Sinir

In [ ]:
# boxplot ile belirlenen aykiri degerlere erismek
aykiri_TF=(my_transaction['TransactionAmt']<my_TransactionAmt_Alt_Sinir)|(my_transaction['TransactionAmt']>my_TransactionAmt_Ust_Sinir)
my_transaction[aykiri_TF].index

In [ ]:
# baskilama ile ust deger sonrasi degerlere ust degeri, alt deger sonrasina ise alt degeri atama
def AykiriDegeriBaskila(deger):
    
    if deger > my_TransactionAmt_Ust_Sinir:
       
        deger=my_TransactionAmt_Ust_Sinir
    elif deger < my_TransactionAmt_Alt_Sinir:
       
        deger=-my_TransactionAmt_Alt_Sinir
    
    return deger

my_transaction['TransactionAmt'] = my_transaction['TransactionAmt'].apply(lambda x: AykiriDegeriBaskila(x))

# HAKANIN YONTEMDENEME

# EKSIKLERI DOLDURMA


In [ ]:
my_transaction.card1.describe()

# Seting train and test back

In [ ]:
df_train, df_test = df[df['isFraud'] != 'test'], df[df['isFraud'] == 2].drop('isFraud', axis=1)